# 06_02_neu: OpenCV Edge & Feature Basics

Ziele:
- Sichere Dtype-Pipeline (uint8/float32) für OpenCV
- Kanten: Canny, Sobel, Laplacian
- Konturen + einfache Features (ORB) mit parameter-tuning

## Einführung
Robuste Edge- und Feature-Detektion mit OpenCV, fokusiert auf fehlerfreie Datentypen, kleine Bilder und interaktive Parameter. ORB dient als lizenzfreundliche Alternative zu SIFT.

**Lernziele**
- Bilder sicher in Graustufen/uint8 überführen, um CV_64F-Fehler zu vermeiden
- Canny, Sobel und Laplacian anwenden und Parameterwirkung erkennen
- Konturen finden, zeichnen und zählen
- ORB-Keypoints berechnen und interpretieren (als SIFT-Alternative)
- Performance steuern: Bildgröße, Kernelgrößen, Thresholds bewusst setzen

In [ ]:
# Setup imports
import os, random
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ipywidgets import interact, fixed

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
random.seed(42)
print("OpenCV", cv2.__version__)


In [ ]:
# Helper: load demo image (fallback synthetic)
def load_demo(path="images/lama.png", size=(400,400)):
    if os.path.exists(path):
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
    else:
        # synthetic stripes if file missing
        xs = np.linspace(0, 1, size[0])
        ys = np.linspace(0, 1, size[1])
        xv, yv = np.meshgrid(xs, ys)
        img = np.stack([
            (xv*255).astype(np.uint8),
            (yv*255).astype(np.uint8),
            ((xv>0.5) ^ (yv>0.5)).astype(np.uint8)*255,
        ], axis=-1)
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    return img

demo = load_demo()
plt.imshow(demo); plt.axis("off"); plt.title("Demo image"); plt.show()


In [ ]:
# Edge detectors with dtype guards
def to_gray_u8(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) if img.ndim == 3 else img
    if gray.dtype != np.uint8:
        gray = np.clip(gray, 0, 255).astype(np.uint8)
    return gray

def run_edges(img, low=80, high=150, ksize=3):
    gray = to_gray_u8(img)
    canny = cv2.Canny(gray, low, high)
    sobelx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=ksize)
    mag = np.sqrt(sobelx**2 + sobely**2)
    mag = np.clip(mag / (mag.max() + 1e-6) * 255, 0, 255).astype(np.uint8)
    lap = cv2.Laplacian(gray, cv2.CV_32F)
    lap = np.clip((lap - lap.min()) / (lap.max() - lap.min() + 1e-6) * 255, 0, 255).astype(np.uint8)
    fig, axes = plt.subplots(1, 4, figsize=(12,3))
    for ax, img_, title in zip(axes, [gray, canny, mag, lap], ["Gray", "Canny", "Sobel mag", "Laplacian"]):
        ax.imshow(img_, cmap="gray")
        ax.set_title(title)
        ax.axis("off")
    plt.tight_layout(); plt.show()

run_edges(demo)

In [ ]:
# Interactive Canny thresholds
@interact(low=(0,200,10), high=(50,300,10), ksize=(3,7,2), img=fixed(demo))
def _tune_edges(low=80, high=150, ksize=3, img=demo):
    if high <= low:
        high = low + 10
    run_edges(img, low, high, int(ksize))


In [ ]:
# Contour detection
def detect_contours(img, thresh1=80, thresh2=150):
    gray = to_gray_u8(img)
    edges = cv2.Canny(gray, thresh1, thresh2)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    result = img.copy()
    cv2.drawContours(result, contours, -1, (255,0,0), 2)
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1); plt.imshow(edges, cmap="gray"); plt.title("Canny"); plt.axis("off")
    plt.subplot(1,2,2); plt.imshow(result); plt.title(f"Contours: {len(contours)}"); plt.axis("off")
    plt.tight_layout(); plt.show()
    return contours

contours = detect_contours(demo)


In [ ]:
# ORB keypoints (license-friendly alternative to SIFT)
def run_orb(img, n_features=300):
    orb = cv2.ORB_create(nfeatures=n_features)
    gray = to_gray_u8(img)
    kps, desc = orb.detectAndCompute(gray, None)
    vis = cv2.drawKeypoints(img, kps, None, color=(0,255,0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
    plt.figure(figsize=(6,4))
    plt.imshow(vis); plt.axis("off"); plt.title(f"ORB keypoints: {len(kps)}")
    plt.show()
    return kps, desc

kps, desc = run_orb(demo, n_features=200)


## Tipps
- Immer vor cvtColor auf uint8/float32 clampen, um CV_64F Fehler zu vermeiden.
- Für schnelle Runs Bild kleiner halten (`size=(256,256)` im Loader).
- ORB ist patentfrei; SIFT nur nutzen, wenn verfügbar (`cv2.SIFT_create`).